In [6]:
import csv
import pandas as pd
import numpy as np
from IPython.display import display

In [7]:
target = 'scan/wheel-equities-1.csv'

In [8]:
# open file
f = open('../' + target, 'r')
data = np.array(list(csv.reader(f)))
f.close()

In [9]:
# convert to dataframe
df = pd.DataFrame(data[:, 1:]).astype(np.float64)
df.index = data[:, 0]
df.columns = [
    'underlying', 'premium', 'roc', 'req_capital', 'be', 'be_moneyness', 
    'prob_be_delta', 'prob_be_iv', 'iv', 'iv_skew', 'drop_dist',
    'udl_year_ret', 'udl_year_ret_r2', 'udl_year_market_corr', 'udl_hist_vol', 
    'udl_iv_percentile', 'udl_hv_percentile', 'above_be_percentile'
]

In [10]:
# filter dataframe
df = df[df['roc'] >= 0.02]

In [11]:
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

# refine columns
df_filt = pd.DataFrame().astype(np.float64)
df_filt['underlying'] = df['underlying']
df_filt['premium'] = df['premium']
df_filt['break_even'] = df['be']
df_filt['safety_score'] = df[['prob_be_delta', 'prob_be_iv']].min(axis=1) * (1 - df['be_moneyness'])
df_filt['be_moneyness'] = df['be_moneyness']
df_filt['drop_dist'] = df['drop_dist']
df_filt['roc'] = df['roc']
df_filt['prob_be_delta'] = df['prob_be_delta']
df_filt['prob_be_iv'] = df['prob_be_iv']
df_filt['ivp_hvp_diff'] = df['udl_iv_percentile'] - df['udl_hv_percentile']
df_filt['iv_skew'] = df['iv_skew']
df_filt['corr'] = df['udl_year_market_corr']
df_filt.index = df.index

In [12]:
# filter symbols
avoid_symbols = ['SRNE', 'INO', 'NVAX', 'MRNA', 'TQQQ', 'FAS', 'TNA', 'SPXL', 'SQQQ', 'UVXY', 'VXX', 'NUGT']
for s in avoid_symbols:
    df_filt = df_filt[~df_filt.index.str.startswith(s)]

In [13]:
# sort dataframe
df_filt_top = df_filt.nlargest(25, 'safety_score')

In [14]:
# output results
pd.set_option('display.max_rows', 25)
display(df_filt_top)

,underlying,premium,break_even,safety_score,be_moneyness,drop_dist,roc,prob_be_delta,prob_be_iv,ivp_hvp_diff,iv_skew,corr
ENPH Nov 20 2020 $47.50 Put,98.28,100.0,46.50,0.499094,0.47314,0.01145,0.02151,0.98149,0.94730,-0.20455,1.54047,0.50835
ENPH Nov 20 2020 $50.00 Put,98.28,120.0,48.80,0.472799,0.49654,0.01094,0.02459,0.97543,0.93910,-0.20455,1.48051,0.50835
GSX Nov 20 2020 $50.00 Put,94.04,150.0,48.50,0.454841,0.51574,0.01053,0.03093,0.94422,0.93925,-0.32273,1.59222,0.21358
AAL Nov 20 2020 $7.00 Put,13.12,15.0,6.85,0.443104,0.52210,0.01039,0.02190,0.97218,0.92719,-0.22272,1.36279,0.45578
CYTK Nov 20 2020 $13.00 Put,29.47,35.0,12.65,0.438661,0.42925,0.01241,0.02767,0.92438,0.76857,-0.65909,0.86186,0.37994
ENPH Nov 20 2020 $55.00 Put,98.28,152.0,53.48,0.419090,0.54416,0.00991,0.02842,0.95939,0.91938,-0.20455,1.41774,0.50835
CYTK Nov 20 2020 $14.00 Put,29.47,60.0,13.40,0.413594,0.45470,0.01185,0.04478,0.91391,0.75847,-0.65909,0.88486,0.37994
GSX Nov 20 2020 $55.00 Put,94.04,200.0,53.00,0.400873,0.56359,0.00949,0.03774,0.92377,0.91857,-0.32273,1.51079,0.21358
BE Nov 20 2020 $10.00 Put,17.92,20.0,9.80,0.397504,0.54688,0.00985,0.02041,0.93302,0.87726,-0.20909,1.10663,0.43121
SPWR Nov 20 2020 $9.00 Put,15.70,19.0,8.81,0.396334,0.56115,0.00954,0.02157,0.95316,0.90312,-0.11818,1.24064,0.52746
